### Reduced-feature models.

We create another price prediction model that doesn't use as many of the dataset features (eg.. community area) since the information is either redundant or difficult to compute in real time for the average user.

In [1]:
import pandas as pd
import numpy as np
import pickle
from Fair_Fare.feature_utils import load_hdf

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline, FeatureUnion, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from joblib import dump, load

import xgboost as xgb
import datetime as dt

from scipy.stats import randint, uniform

# Plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
mpl.rc('axes', labelsize=18)
mpl.rc('xtick', labelsize=16)
mpl.rc('ytick', labelsize=16)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

### TNP Dataset:

Parameter tuning

In [2]:
tnp = load_hdf('data/tnp_train.h5',.02)
tnp.drop(['Pickup_Community_Area', 'Dropoff_Community_Area','l2_dist_km','x_dist_km','y_dist_km','bearing'], axis = 1, inplace = True)
tnp.head()
y = tnp["Final_Fare"].copy()
X = tnp.drop(["Final_Fare"], axis = 1)

In [7]:
tnp = load_hdf('data/tnp_train.h5',.01)
# drop extra features
tnp.drop(['Pickup_Community_Area', 'Dropoff_Community_Area','l2_dist_km','x_dist_km','y_dist_km','bearing'], axis = 1, inplace = True)

y = tnp["Final_Fare"].copy()
X = tnp.drop(["Final_Fare"], axis = 1)


X_train, Xv, y_train, yv = train_test_split(X, y, test_size=0.2, random_state=42)

dvalid = xgb.DMatrix(Xv.values, label=yv.values)
dtrain = xgb.DMatrix(X_train.values, label=y_train.values)

param_dist = {'max_depth': randint(low=4, high=20),
              'n_estimators': randint(low=20, high=400),
              'min_child_weight':randint(low=5, high=200),
              'eta':uniform(0.15,.2),
              'colsample_bytree':uniform(0.2,.4),
              'subsample': uniform(0.4,.5),
              'lambda': uniform(0.5,3.)
             }

xgb_model = xgb.XGBRegressor({
              'booster' : 'gbtree', 'eval_metric': 'rmse','silent': 1, 'objective': 'reg:squarederror'})
clf =  RandomizedSearchCV(xgb_model,
                   param_dist, verbose=1,n_jobs = 6,n_iter=100, cv=5, scoring='neg_mean_squared_error')
clf.fit(X.values,y.values)
print(clf.best_score_)
print(clf.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    2.1s
[Parallel(n_jobs=6)]: Done 409 tasks      | elapsed:   12.9s
[Parallel(n_jobs=6)]: Done 489 out of 500 | elapsed:   15.3s remaining:    0.3s
[Parallel(n_jobs=6)]: Done 500 out of 500 | elapsed:   15.6s finished
/home/isaac/anaconda3/envs/dinc_proj/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[08:36:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
-10.699642221720152
{'colsample_bytree': 0.5830943566473055, 'eta': 0.29138688164777615, 'lambda': 1.3572576723966736, 'max_depth': 18, 'min_child_weight': 10, 'n_estimators': 247, 'subsample': 0.5195772003626937}


##### Full model:

In [6]:
X = load_hdf('data/tnp_train.h5',10)
X.drop(['Pickup_Community_Area', 'Dropoff_Community_Area','l2_dist_km','x_dist_km','y_dist_km','bearing'], axis = 1, inplace = True)

y = X["Final_Fare"].copy()
X.drop(["Final_Fare"], axis = 1, inplace = True)


X_train, Xv, y_train, yv = train_test_split(X, y, test_size=0.2, random_state=42)

dvalid = xgb.DMatrix(Xv.values, label=yv.values,feature_names = Xv.columns)
dtrain = xgb.DMatrix(X_train.values, label=y_train.values, feature_names = X_train.columns)

watchlist = [(dtrain, 'train'), (dvalid, 'valid')]


xgb_pars = {'colsample_bytree': 0.439881349485437, 'eta': 0.2913248799253698, 'lambda': 1.0784417148624983, 'max_depth': 5, 
            'min_child_weight': 61, 'n_estimators': 131, 'subsample': 0.8144003761476369, 'nthread': 7, 
            'booster' : 'gbtree', 'silent': 1,
            'eval_metric': 'rmse', 'objective': 'reg:squarederror'}


t0 = dt.datetime.now()
model = xgb.train(xgb_pars, dtrain, 500, watchlist, early_stopping_rounds=50,
                  maximize=False, verbose_eval=2)

t1 = dt.datetime.now()
print('Modeling RMSLE %.5f' % model.best_score)
print('Training time: %i seconds' % (t1 - t0).seconds)

#model.save_model("tnp_xgb_full")

# this is a way of saving metadata like feature names to the model
if hasattr(model, 'feature_names'): model.set_attr(feature_names = '|'.join(model.feature_names))

model.save_model("tnp_xgb_full_reduced_params")

[0]	train-rmse:10.3148	valid-rmse:10.2749
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[1]	train-rmse:8.17851	valid-rmse:8.13749
[2]	train-rmse:6.49763	valid-rmse:6.45791
[3]	train-rmse:5.24322	valid-rmse:5.20372
[4]	train-rmse:4.50248	valid-rmse:4.46311
[5]	train-rmse:4.11955	valid-rmse:4.08209
[6]	train-rmse:3.82896	valid-rmse:3.79323
[7]	train-rmse:3.58179	valid-rmse:3.54614
[8]	train-rmse:3.50313	valid-rmse:3.46806
[9]	train-rmse:3.42995	valid-rmse:3.3951
[10]	train-rmse:3.4105	valid-rmse:3.37678
[11]	train-rmse:3.39192	valid-rmse:3.35882
[12]	train-rmse:3.37445	valid-rmse:3.34226
[13]	train-rmse:3.35115	valid-rmse:3.31985
[14]	train-rmse:3.3245	valid-rmse:3.29334
[15]	train-rmse:3.14238	valid-rmse:3.10958
[16]	train-rmse:3.12631	valid-rmse:3.09346
[17]	train-rmse:3.0273	valid-rmse:2.99245
[18]	train-rmse:3.01966	valid-rmse:2.98514
[19]	train-rmse:3.01429	valid-rmse:2.98007
[20]	trai

### Taxi Dataset:

In [ ]:
taxi = load_hdf('data/taxi_train.h5',1)
# drop extra features
taxi.drop(['Pickup_Community_Area', 'Dropoff_Community_Area','l2_dist_km','x_dist_km','y_dist_km','bearing'], axis = 1, inplace = True)

y = taxi["Final_Fare"].copy()
X = taxi.drop(["Final_Fare"], axis = 1)


X_train, Xv, y_train, yv = train_test_split(X, y, test_size=0.2, random_state=42)

dvalid = xgb.DMatrix(Xv.values, label=yv.values)
dtrain = xgb.DMatrix(X_train.values, label=y_train.values)


# parameter search
param_dist = {'max_depth': randint(low=4, high=20),
              'n_estimators': randint(low=20, high=400),
              'min_child_weight':randint(low=5, high=200),
              'eta':uniform(0.15,.2),
              'colsample_bytree':uniform(0.2,.4),
              'subsample': uniform(0.4,.5),
              'lambda': uniform(0.5,3.)
             }
xgb_model = xgb.XGBRegressor({
              'booster' : 'gbtree', 'eval_metric': 'rmse','silent': 1, 'objective': 'reg:squarederror'})
clf =  RandomizedSearchCV(xgb_model,
                   param_dist, verbose=1,n_jobs = 7,n_iter=200, cv=4, scoring='neg_mean_squared_error')
clf.fit(X.values,y.values)
print(clf.best_score_)
print(clf.best_params_)

Fitting 4 folds for each of 200 candidates, totalling 800 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:  5.7min
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed: 26.6min
[Parallel(n_jobs=7)]: Done 436 tasks      | elapsed: 65.1min


In [14]:
X = load_hdf('data/taxi_train.h5',.1)

y = X["Final_Fare"].copy()
X.drop(['Pickup_Community_Area', 'Dropoff_Community_Area','l2_dist_km','x_dist_km','y_dist_km','bearing'], axis = 1, inplace = True)
X.drop(["Final_Fare"], axis = 1, inplace = True)


X_train, Xv, y_train, yv = train_test_split(X, y, test_size=0.2, random_state=42)

dvalid = xgb.DMatrix(Xv.values, label=yv.values,feature_names = Xv.columns)
dtrain = xgb.DMatrix(X_train.values, label=y_train.values, feature_names = X_train.columns)

watchlist = [(dtrain, 'train'), (dvalid, 'valid')]



xgb_pars = {'colsample_bytree': 0.4659175029639101, 'eta': 0.30900240687858116, 
            'lambda': 3.335261321413178, 'max_depth': 7, 'min_child_weight': 8, 
            'n_estimators': 355, 'subsample': 0.7367235778794784, 'nthread': 6, 
            'booster' : 'gbtree', 'silent': 1,
            'eval_metric': 'rmse', 'objective': 'reg:squarederror'}


t0 = dt.datetime.now()
model = xgb.train(xgb_pars, dtrain, 500, watchlist, early_stopping_rounds=50,
                  maximize=False, verbose_eval=10)

t1 = dt.datetime.now()
print('Modeling RMSLE %.5f' % model.best_score)
print('Training time: %i seconds' % (t1 - t0).seconds)

# this is a way of saving metadata like feature names to the model
if hasattr(model, 'feature_names'): model.set_attr(feature_names = '|'.join(model.feature_names))

model.save_model("taxi_xgb_full_reduced_params")

[0]	train-rmse:13.3396	valid-rmse:12.9789
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[10]	train-rmse:1.83602	valid-rmse:1.93658
[20]	train-rmse:1.50854	valid-rmse:1.72833
[30]	train-rmse:1.42197	valid-rmse:1.68055
[40]	train-rmse:1.35854	valid-rmse:1.66849
[50]	train-rmse:1.31555	valid-rmse:1.65325
[60]	train-rmse:1.28148	valid-rmse:1.64402
[70]	train-rmse:1.24424	valid-rmse:1.65307
[80]	train-rmse:1.21823	valid-rmse:1.66437
[90]	train-rmse:1.18152	valid-rmse:1.64327
[100]	train-rmse:1.15291	valid-rmse:1.65473
[110]	train-rmse:1.11594	valid-rmse:1.65255
Stopping. Best iteration:
[62]	train-rmse:1.27332	valid-rmse:1.64322

Modeling RMSLE 1.64322
Training time: 0 seconds


In [15]:
model.feature_names

['Trip_Seconds',
 'Trip_Miles',
 'Pickup_Centroid_Latitude',
 'Pickup_Centroid_Longitude',
 'Dropoff_Centroid_Latitude',
 'Dropoff_Centroid_Longitude',
 'vel_mph',
 'bAirport',
 'day_of_wk',
 'hour']